Author: Kyle Herbruger
Date: 10/17/2023

Most of the hand tracking code comes from: https://www.section.io/engineering-education/creating-a-hand-tracking-module/

This program tracks the users on camera hands, and saves the data to two .csv files.
Will write hand location as 0 in the case of no hand being detected to keep time scale accurate.

In [25]:
import cv2
import mediapipe as mp
import numpy as np
import sys
import matplotlib.pyplot as plt

# Used to convert protobuf message to a dictionary. 
from google.protobuf.json_format import MessageToDict

In [26]:
class handTracker():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5,modelComplexity=1,trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.modelComplex = modelComplexity
        self.trackCon = trackCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,self.modelComplex,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils    
        
    def handsFinder(self,image,draw=True):
        imageRGB = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imageRGB)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:

                if draw:
                    self.mpDraw.draw_landmarks(image, handLms, self.mpHands.HAND_CONNECTIONS)
        return image
    
    def positionFinder(self,image, handNo=0, draw=True):
        lmlist = []
        if self.results.multi_hand_landmarks:
            Hand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(Hand.landmark):
                h,w,c = image.shape
                cx,cy,cz = int(lm.x*w), int(lm.y*h), int(1000*lm.z)
                lmlist.append([id,cx,cy,cz])
            # Draws out pink circle on pinky tip
            if draw:
                cv2.circle(image,(cx,cy), 15 , (255,0,255), cv2.FILLED)

        return lmlist

In [31]:
def main():
    fileLoc = './LR_CSV/binarized'
    #cap = cv2.VideoCapture(0)
    cap = cv2.VideoCapture("./TestVid/tv2.mp4")
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(fps)
    vidLen = 45
    tracker = handTracker()
    
    timer = int(fps * vidLen)
    t = timer
    lmListLeft_CSV = []
    lmListRight_CSV = []
    
    bFrames = []
    totalSkippedFrames = 0
    mtFrames = 0
    frSpree = 0
    # loop to repeatedly scan webcam and process video
    while (t > 0):
        success,image = cap.read()
        #cv2.imshow("frame", image)
        image = cv2.flip(image, 1) 
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        image = tracker.handsFinder(image)
        # Original imp: lmListLeft = tracker.positionFinder(image,0)

        # Check if hand detected at all
        if tracker.results.multi_handedness:
            frSpree = frSpree + 1
            if mtFrames != 0:
                print(f'Skipped {mtFrames} Frames...')
                totalSkippedFrames = totalSkippedFrames + mtFrames
                mtFrames = 0
                bFrames.append(mtFrames)
            if len(tracker.results.multi_handedness) == 2: # Check if both hands detected
                lmListLeft_CSV.append(tracker.positionFinder(image,0))
                lmListRight_CSV.append(tracker.positionFinder(image,1))
#                 print('LR')
            else: # Determine which hand is detected
                for i in tracker.results.multi_handedness:
                    label = MessageToDict(i)['classification'][0]['label'] 
                    if label == 'Left':  # Left hand 
                        lmListLeft_CSV.append(tracker.positionFinder(image,0))
#                         print('L')
                    if label == 'Right': # Right hand
                        lmListRight_CSV.append(tracker.positionFinder(image,0))
#                         print('R')
        else: 
            if (frSpree > 1):
                print(f'Did {frSpree} frames in a row...')
            frSpree = 0
            mtFrames = mtFrames + 1
        # End of handedness detection    
        #cv2.imshow("Video",image)
        cv2.waitKey(1)
        t = t - 1
        if (t % 300 == 0):
            print(((timer-t)*100)/timer, '%')
    
    cap.release()
    cv2.destroyAllWindows()

    npArrLeft = np.zeros((timer, 21, 4))
    npArrRight = np.zeros((timer, 21, 4))
    
    for ii in range(len(lmListLeft_CSV)):
        for ib in range(len(lmListLeft_CSV[ii])):
            npArrLeft[ii, ib] = np.array(lmListLeft_CSV[ii][ib])
    for ii in range(len(lmListRight_CSV)):
        for ib in range(len(lmListRight_CSV[ii])):
            npArrRight[ii, ib] = np.array(lmListRight_CSV[ii][ib])
            
    npArrLeft = npArrLeft.astype(int)
    npArrRight = npArrRight.astype(int)
    
    np.savetxt('handIDNum.csv', npArrLeft[:,:,0], delimiter=',')
    np.savetxt(f'{fileLoc}LeftX.csv', npArrLeft[:,:,1], delimiter=',', fmt='%d')
    np.savetxt(f'{fileLoc}LeftY.csv', npArrLeft[:,:,2], delimiter=',', fmt='%d')
    np.savetxt(f'{fileLoc}LeftZ.csv', npArrLeft[:,:,3], delimiter=',', fmt='%d')

    np.savetxt(f'{fileLoc}RightX.csv', npArrRight[:,:,1], delimiter=',', fmt='%d')
    np.savetxt(f'{fileLoc}RightY.csv', npArrRight[:,:,2], delimiter=',', fmt='%d')
    np.savetxt(f'{fileLoc}RightZ.csv', npArrRight[:,:,3], delimiter=',', fmt='%d')
    
    print('Lost Ratio: ', (totalSkippedFrames/timer)*100, '%')
    # 86% for gray run 1
    # 86 for gr2
    plt.scatter(len(bFrames), bFrames)
    plt.show()

In [32]:
if __name__ == "__main__":
    main()
    print("Done XD")

59.701
Skipped 42 Frames...
Did 9 frames in a row...
Skipped 127 Frames...
Did 4 frames in a row...
Skipped 37 Frames...
Did 3 frames in a row...
Skipped 2 Frames...
10.647803425167535 %
Skipped 120 Frames...
Skipped 34 Frames...
Skipped 5 Frames...
Skipped 1 Frames...
Skipped 1 Frames...
Did 6 frames in a row...
Skipped 16 Frames...
Skipped 28 Frames...
Did 2 frames in a row...
Skipped 73 Frames...
Skipped 1 Frames...
Skipped 1 Frames...
Skipped 1 Frames...
Did 3 frames in a row...
Skipped 1 Frames...
Did 4 frames in a row...
21.816827997021594 %
Skipped 108 Frames...
Did 2 frames in a row...
Skipped 3 Frames...
Skipped 8 Frames...
Skipped 4 Frames...
Skipped 1 Frames...
Skipped 2 Frames...
Skipped 23 Frames...
Skipped 18 Frames...
Skipped 2 Frames...
Did 15 frames in a row...
Skipped 131 Frames...
Skipped 11 Frames...
32.98585256887565 %
Skipped 26 Frames...
Skipped 225 Frames...
Did 9 frames in a row...
Skipped 1 Frames...
Did 6 frames in a row...
Skipped 40 Frames...
44.15487714072

TypeError: scatter() missing 1 required positional argument: 'y'

NameError: name 'bFrames' is not defined